# Analysis - zBrains outputs at 3T and 7T in epilepsy
1. zBrain/wBrain (surface)  
    a. Histograms of vertex wise scores  
        i. sub-comparisons with different smoothing kernels  
    b. Quantifying extreme vertex groups  
        i. number of identified abnormal areas  
        ii. size of each abnormal area (number of adjacent extreme vertices)  
2. Brainstats (surface)  
    a. t-scores for 3T and 7T  
    b. cohen's D map between 3T and 7T images  


## 0. Initialize

In [ ]:
import os
import sys
import importlib
import tTsTGrpUtils as tsutil
import utils_plots as uplots

dl_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/04d_dl_maps_19Oct2025-153909.pkl"
dl = tsutil.loadPickle(dl_pth)
tsutil.print_dict(dl, df_print=False)


In [ ]:
idx = 2
item = 'df_demo'
df = dl[idx][item]
#df = tsutil.loadPickle(dl[idx][item])
df

In [ ]:
import nibabel as nib
import pandas as pd
import numpy as np
from brainspace.plotting import plot_hemispheres
from brainspace.mesh.mesh_io import read_surface
from hippunfold_toolbox import plotting as plot_hu
importlib.reload(tsutil)
importlib.reload(uplots)
# 
# 46,47,
item_idx = [44,5,20,21,46,47, 52, 53]

idx_3T = ["UID0018_PX071_04","UID0027_PX168_01", "UID0030_PX148_01", "UID0023_PX173_01"]
idx_7T = ["UID0018_PNE001_01", "UID0027_PNE010_a1", "UID0030_PNE013_a1", "UID0023_PNE006_a1"]
save_maps = "/host/verges/tank/data/daniel/3T7T/qualitative_pics/maps"

def get_mesh(region, surface, micapipe = "/data_/mica1/01_programs/micapipe-v0.2.0", hipp_surfaces = "/host/verges/tank/data/daniel/3T7T/z/code/analyses/resources/", inflated = True):
    if region in ["ctx", "cortex"]:
        if surface == 'fsLR-5k':
            if inflated == True:
                # Load fsLR 5k inflated
                surf_lh = read_surface(micapipe + '/surfaces/fsLR-5k.L.inflated.surf.gii', itype='gii')
                surf_rh = read_surface(micapipe + '/surfaces/fsLR-5k.R.inflated.surf.gii', itype='gii')
            else:
                # Load fsLR 5k
                surf_lh = read_surface(micapipe + '/surfaces/fsLR-5k.L.surf.gii', itype='gii')
                surf_rh = read_surface(micapipe + '/surfaces/fsLR-5k.R.surf.gii', itype='gii')
        elif surface == 'fsLR-32k':
            if inflated == True:
                # Load fsLR 32k inflated
                surf_lh = read_surface(micapipe + '/surfaces/fsLR-32k.L.inflated.surf.gii', itype='gii')
                surf_rh = read_surface(micapipe + '/surfaces/fsLR-32k.R.inflated.surf.gii', itype='gii')
            else:
                # Load fsLR 32k
                surf_lh = read_surface(micapipe + '/surfaces/fsLR-32k.L.surf.gii', itype='gii')
                surf_rh = read_surface(micapipe + '/surfaces/fsLR-32k.R.surf.gii', itype='gii')
        else:
            raise ValueError(f"Surface {surface} not recognized. Use 'fsLR-5k' or 'fsLR-32k'.")
        
    elif region in ["hip", "hipp", "hippocampus"]: # only have templates for midthickness
        if surface in ['0p5mm','den-0p5mm']: # use hippunfold toolbox
            surf_lh = read_surface(hipp_surfaces + 'tpl-avg_space-canonical_den-0p5mm_label-hipp_midthickness_L.surf.gii', itype='gii')
            surf_rh = read_surface(hipp_surfaces + 'tpl-avg_space-canonical_den-0p5mm_label-hipp_midthickness_R.surf.gii', itype='gii')

            # unfolded
            surf_lh_unf = read_surface(hipp_surfaces + 'tpl-avg_space-unfold_den-0p5mm_label-hipp_midthickness.surf.gii', itype='gii')
            surf_rh_unf = read_surface(hipp_surfaces + 'tpl-avg_space-unfold_den-0p5mm_label-hipp_midthickness.surf.gii', itype='gii')

    return surf_lh, surf_rh

def hist(z_lh, z_rh, title, save_pth, d=None, mn = True):
    import matplotlib.pyplot as plt
    z_lh, z_rh = z_map[:len(pt_l)], z_map[len(pt_l):]
    import scipy.stats as stats
    
    # Create smoothed histograms using kernel density estimation
    z_lh_clean = z_lh[~np.isnan(z_lh)]
    z_rh_clean = z_rh[~np.isnan(z_rh)]
    
    if len(z_lh_clean) > 0:
        kde_lh = stats.gaussian_kde(z_lh_clean)
        x_range = np.linspace(min(z_lh_clean.min(), z_rh_clean.min()), 
                             max(z_lh_clean.max(), z_rh_clean.max()), 200)
        plt.plot(x_range, kde_lh(x_range), label='L', linewidth=2)
    
    if len(z_rh_clean) > 0:
        kde_rh = stats.gaussian_kde(z_rh_clean)
        x_range = np.linspace(min(z_lh_clean.min(), z_rh_clean.min()), 
                             max(z_lh_clean.max(), z_rh_clean.max()), 200)
        plt.plot(x_range, kde_rh(x_range), label='R', linewidth=2)
    if d is not None:
        # add annotation
        plt.annotate(f"d = {d:.2f}", xy=(0.7, 0.9), xycoords='axes fraction', fontsize=12,
                     bbox=dict(boxstyle="round,pad=0.3", fc="yellow", ec="black", lw=1))
    if mn:
        plt.annotate(f"mn_L = {z_lh.mean():.2f}\nmn_R = {z_rh.mean():.2f}", xy=(0.7, 0.75), xycoords='axes fraction', fontsize=12,
                     bbox=dict(boxstyle="round,pad=0.3", fc="yellow", ec="black", lw=1))
    ax = plt.gca()
    ax.get_yaxis().set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.legend()
    plt.title(title)
    plt.savefig(save_pth)
    plt.close()

def smoothMaps(sub, ses, ft, lbl, region, surf, root_dir, root_mp, root_deriv, study):
    if ft == 'thickness':
        surf_pth = tsutil.get_surf_pth(root=f"{root_deriv}", sub=sub, ses=ses, lbl='midthickness', surf = surf)
    else:
        surf_pth = tsutil.get_surf_pth(root=f"{root_deriv}", sub=sub, ses=ses, lbl=lbl, surf = surf)
    
    out_name_base = f"map-{sub}-{ses}_{ft}_{lbl}_{region}_{surf}"
    
    if region == 'cortex':
        smth = 10
    else:
        smth = 2

    final_out_L = f"{save_maps}/{out_name_base}_L-smth{smth}mm.func.gii"
    final_out_R = f"{save_maps}/{out_name_base}_R-smth{smth}mm.func.gii"

    if tsutil.chk_pth(final_out_L) and tsutil.chk_pth(final_out_R):
        #print(f"Found existing smoothed maps for {ctrl}, skipping processing.")
        lh = nib.load(final_out_L).darrays[0].data
        rh = nib.load(final_out_R).darrays[0].data
        return lh, rh

    if region == 'cortex':
        map_pth = tsutil.get_map_pth(root = root_dir, deriv_fldr="micapipe_v0.2.0", sub = sub, ses = ses, feature = ft,
                        label = lbl, surf = surf)
        
    else:
        if ft == 'thickness':
            map_pth = tsutil.get_surf_pth(root_dir + "/derivatives/hippunfold_v1.3.0/hippunfold", sub=sub, ses=ses, lbl='thickness', surf=surf)
            map_pth_lh = map_pth[0]
            map_pth_rh = map_pth[1]
        else:
            vol_pth =tsutil.get_mapVol_pth(root_mp, sub, ses, study, ft)
            print(vol_pth)
            out_name = f"{save_maps}/{out_name_base}.func.gii"
            map_pth_lh = tsutil.map(vol_pth, surf_pth[0], out_name)
            map_pth_rh = tsutil.map(vol_pth, surf_pth[1], out_name)
    
    #print(map_pth)

    surf_pth_lh = surf_pth[0]
    surf_pth_rh = surf_pth[1]

    lh_smth_pth = tsutil.smooth_map(surf = surf_pth_lh, map = map_pth_lh, out_name = final_out_L, kernel = smth) 
    rh_smth_pth = tsutil.smooth_map(surf = surf_pth_rh, map = map_pth_rh , out_name = final_out_R, kernel = smth)

    lh = nib.load(lh_smth_pth).darrays[0].data
    rh = nib.load(rh_smth_pth).darrays[0].data

    return lh, rh

for i in item_idx:
    itm = dl[i]
    study = itm['study']
    region = itm['region']
    surf = itm['surf']
    lbl = itm['label']
    ft = itm['feature']
    
    if study == 'MICs':
        pt_idx = idx_3T
        id_col = "MICS_ID"
        study_code = "3T"
        root_dir = "/data/mica3/BIDS_MICs"
        cbar_pos = 'right'
    else:
        pt_idx = idx_7T
        id_col = "PNI_ID"
        study_code = "7T"
        root_dir = "/data/mica3/BIDS_PNI"
        cbar_pos = 'left'

    root_mp = f"{root_dir}/derivatives/micapipe_v0.2.0"
    if region in ['hippocampus', 'hip']:
        root_deriv = f"{root_dir}/derivatives/hippunfold_v1.3.0/hippunfold"
    else:
        root_deriv = root_mp
    
    df = itm['df_maps']
    if isinstance(df, str):
        df = tsutil.loadPickle(df)

    df_demo = itm['df_demo']
    idx_ctrl = df_demo[(df_demo['grp'] == 'CTRL') & (df_demo['study'] == study_code)]
    idx_ctrl['UID_ID_SES'] = idx_ctrl['UID'] + "_" + idx_ctrl[id_col] + "_" + idx_ctrl['SES']
    idx_ctrl = idx_ctrl['UID_ID_SES'].values.tolist()

    # get maps of controls
    df_maps_ctrl = pd.DataFrame()

    for ctrl in idx_ctrl:
        parts = ctrl.split('_')
        sub = parts[1]  # Extract ID from UID_ID_SES format
        ses = parts[2]  # Extract SES from UID_ID_SES format

        lh, rh = smoothMaps(sub=sub, ses=ses, ft=ft, lbl=lbl, region=region, surf=surf,
                             root_dir=root_dir, root_mp=root_mp, root_deriv=root_deriv,
                             study=study)
        
        combined_data = np.concatenate([lh, rh]) # rh is the second half of the data
        df_maps_ctrl[ctrl] = combined_data

    print(f"df_maps_ctrl <{df_maps_ctrl.shape}>: {df_maps_ctrl.columns.tolist()}")
    #print(df_maps_ctrl.head())
    
    mean_ctrl = df_maps_ctrl.mean(axis=1)
    mn_lh, mn_rh = mean_ctrl[:len(lh)], mean_ctrl[len(lh):]
    std_ctrl = df_maps_ctrl.std(axis=1)
    std_lh, std_rh = std_ctrl[:len(lh)], std_ctrl[len(lh):]
    #print(mean_ctrl.head())
    #print(std_ctrl.head())
    
    print(f"mean: {mean_ctrl.shape}")
    print(f"std: {std_ctrl.shape}")

    #lh_np = lh.flatten()
    #rh_np = rh.to_numpy().flatten()
    # save mn as surface mesh
    surf_lh, surf_rh = get_mesh(region=region, surface=surf)
    data_mn = np.concatenate([mn_lh.values.flatten(), mn_rh.values.flatten()])
    save_name_mn = f"/host/verges/tank/data/daniel/3T7T/qualitative_pics/ctrlMEAN_{itm['feature']}_{itm['study']}_{itm['region']}_{itm['surf']}_{itm['label']}_{itm['smth']}mm.png"
    if ft == 't1map':
        min_val = 1000
        max_val = 4500
    else:git com
        min_val = 0
        max_val = 4.5
    plot_hemispheres(
                    surf_lh, surf_rh, array_name=data_mn, 
                    size=(900, 250), color_bar='right', zoom=1.25, embed_nb=True, interactive=False, share='both',
                    nan_color=(0, 0, 0, 1), color_range=(min_val,max_val), cmap='Blues', transparent_bg=True, 
                    screenshot=True, filename=save_name_mn,
                    #, label_text = lbl_text
                )
    
    print(f"Saved mean figure to {save_name_mn}")

    data_std = np.concatenate([std_lh.values.flatten(), std_rh.values.flatten()])
    save_name_std = f"/host/verges/tank/data/daniel/3T7T/qualitative_pics/ctrlStd_{itm['feature']}_{itm['study']}_{itm['region']}_{itm['surf']}_{itm['label']}_{itm['smth']}mm.png"
    plot_hemispheres(
                    surf_lh, surf_rh, array_name=data_std, 
                    size=(900, 250), color_bar='right', zoom=1.25, embed_nb=True, interactive=False, share='both',
                    nan_color=(0, 0, 0, 1), color_range=(0,0.5), cmap='Blues', transparent_bg=True, 
                    screenshot=True, filename=save_name_std,
                    #, label_text = lbl_text
                )
    print(f"Saved std figure to {save_name_std}")
   
    for pt in pt_idx:
        sub = pt.split('_')[1]
        ses = pt.split('_')[2]

        pt_l, pt_r = smoothMaps(sub=sub, ses=ses, ft=ft, lbl=lbl, region=region, surf=surf,
                             root_dir=root_dir, root_mp=root_mp, root_deriv=root_deriv,
                             study=study)
        mn_pt_l, mn_pt_r = pt_l.mean(), pt_r.mean()
        std_pt_l, std_pt_r = pt_l.std(), pt_r.std()
        n_l = n_r = len(pt_l)
        pooled_std = (((n_l - 1) * std_pt_l**2 + (n_r - 1) * std_pt_r**2) / (n_l + n_r - 2))**0.5
        d = (mn_pt_l - mn_pt_r) / pooled_std
        print(f"D: {d}")
        
        #print(f"ctrl_mn: {mean_ctrl[:20]}")
        #print(f"pt_l: {pt_l[:20]}")
        map_pt =  np.concatenate([pt_l, pt_r])
        #print(f"map_pt:\n{map_pt[:20]}")
        z_map = (map_pt - mean_ctrl) / std_ctrl
        #print(f"z_map:\n{z_map[:20]}")
        # plot histogram of z_map
        z_lh, z_rh = z_map[:len(pt_l)], z_map[len(pt_l):]
        
        save_name_pt = f"/host/verges/tank/data/daniel/3T7T/qualitative_pics/{pt}_{itm['region']}_{itm['feature']}_{itm['study']}_{itm['surf']}_{itm['label']}_{itm['smth']}mm_zmap.png"
        
        if region == 'cortex':
            z_lh, z_rh = tsutil.apply_midMask(z_lh, z_rh, surf = itm['surf'])
            data_pt = np.concatenate([z_lh.values.flatten(), z_rh.values.flatten()])
            plot_hemispheres(
                surf_lh, surf_rh, array_name=data_pt, 
                size=(900, 250), color_bar='right', zoom=1.25, embed_nb=True, interactive=False, share='both',
                nan_color=(0, 0, 0, 1), color_range=(-4,4), cmap='seismic', transparent_bg=True, 
                screenshot=True, filename=save_name_pt,
                #, label_text = lbl_text
            )
        else:
            lh_array = np.asarray(z_lh).flatten()
            rh_array = np.asarray(z_rh).flatten()
            data_pt = np.column_stack([z_lh, z_rh])
            data_pt = data_pt[:, :, np.newaxis]
            plot_hu.surfplot_canonical_foldunfold(data_pt, tighten_cwindow=True, 
                                                  labels='hipp', color_bar='right', 
                                                  color_range = (-4,4), cmap='seismic', share = True,
                                                 screenshot=True, filename = save_name_pt)
            
        print(f"Saved z-map figure to {save_name_pt}")

        hist(z_lh, z_rh, title = f"{pt} - {study} {region} {ft} {surf}", save_pth=f"/host/verges/tank/data/daniel/3T7T/qualitative_pics/{pt}_{itm['feature']}_{itm['study']}_{itm['region']}_{itm['surf']}_{itm['label']}_{itm['smth']}mm_zmap_hist.png", d = d, mn = [mn_pt_l, mn_pt_r])
        

In [ ]:
# read these in
a = nib.load("/data/mica3/BIDS_MICs/derivatives/hippunfold_v1.3.0/hippunfold/sub-HC129/ses-01/surf/sub-HC129_ses-01_hemi-L_space-T1w_den-0p5mm_label-hipp_midthickness.surf.gii").darrays[0].data

b = nib.load("/data/mica3/BIDS_MICs/derivatives/hippunfold_v1.3.0/hippunfold/sub-HC129/ses-01/surf/sub-HC129_ses-01_hemi-L_space-T1w_den-0p5mm_label-hipp_midthickness.surf.gii").darrays[0].data
print(a.shape, b.shape)


In [ ]:
# make gifs from lower and higher res image
import nibabel as nib
import numpy as np
import cv2
import os

# -------------------------------
# CONFIGURATION
# -------------------------------
img = "7T"
dpi = 500
fps = 10
slice_step = 1  # Use >1 to skip slices for fewer frames

if img == "7T":
    input_path = "/data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE017/sub-PNE017_ses-a1_space-nativepro_T1w_brain.nii.gz"  
    output_folder = "/host/verges/tank/data/daniel/3T7T/z/coregVols/PNE017_PX174/7T_pngs/"
    output_video = "/host/verges/tank/data/daniel/3T7T/z/coregVols/PNE017_PX174/7T_pngs/coronal_animation.mp4"
    start_slice = 61
    end_slice = 381
elif img == "3T":
    input_path = "/host/verges/tank/data/daniel/3T7T/z/coregVols/PNE017_PX174/PX174a_0000.nii.gz"   # Replace with your file
    output_folder = "/host/verges/tank/data/daniel/3T7T/z/coregVols/PNE017_PX174/3T_pngs/"
    output_video = "/host/verges/tank/data/daniel/3T7T/z/coregVols/PNE017_PX174/3T_pngs/coronal_animation.mp4"
    start_slice = 61
    end_slice = 330
    

# -------------------------------
# LOAD VOLUME
# -------------------------------
img = nib.load(input_path)
data = img.get_fdata()
norm_data = (data - np.min(data)) / (np.max(data) - np.min(data))
norm_data = (norm_data * 255).astype(np.uint8)

# Coronal plane => slice along axis 1
n_slices = data.shape[1]
height, width = norm_data.shape[0], norm_data.shape[2]

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height), isColor=False)

for i in range(0, n_slices, slice_step):
    # Extract coronal slice and rotate for correct orientation
    slice_img = np.rot90(norm_data[:, i, :])
    
    # Convert to BGR for video writing if needed (here grayscale video, so no)
    video_writer.write(slice_img)

video_writer.release()
print(f"MP4 video saved to {output_video}")

In [ ]:
# go through volumes, sweep with yellow line for transition

import nibabel as nib
import numpy as np
import cv2

# Parameters
path_3T = "/host/verges/tank/data/daniel/3T7T/z/coregVols/PNE017_PX174/PX174a_0000_resampled.nii.gz"
tT_min = 473
tT_max = 5254
path_7T = "/data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE017/ses-a1/anat/sub-PNE017_ses-a1_space-nativepro_T1w.nii.gz"
sT_min = -5
sT_max = 79
output_video = "/host/verges/tank/data/daniel/3T7T/z/coregVols/PNE017_PX174/PX174-PNE017.mp4"
fps = 30
transition_begin = 1/3
transition_frames = 30
line_width = 3
line_color = (0, 255, 255)  # Yellow in BGR

def apply_intensity_window(image, window_min, window_max):
    clipped = np.clip(image, window_min, window_max)
    scaled = ((clipped - window_min) / (window_max - window_min) * 255.0).astype(np.uint8)
    return scaled

def vol_to_bgr_frames_with_window(vol, wmin, wmax, n_slices):
    frames = []
    for i in range(n_slices):
        slice_img = np.rot90(vol[:, i, :])
        slice_windowed = apply_intensity_window(slice_img, wmin, wmax)
        bgr_img = cv2.cvtColor(slice_windowed, cv2.COLOR_GRAY2BGR)
        frames.append(bgr_img)
    return frames

# Load volumes
img_a = nib.load(path_3T)
img_b = nib.load(path_7T)

vol_a = img_a.get_fdata()
vol_b = img_b.get_fdata()
assert vol_a.shape == vol_b.shape, "Volumes must have the same shape"

n_slices = vol_a.shape[1]
height, width = np.rot90(vol_a[:, 0, :]).shape

frames_a = vol_to_bgr_frames_with_window(vol_a, tT_min, tT_max, n_slices)
frames_b = vol_to_bgr_frames_with_window(vol_b, sT_min, sT_max, n_slices)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height), isColor=True)

transition_idx = int(transition_begin * n_slices)

# Part 1: Show volume A slices up to transition_idx
for i in range(transition_idx):
    frame = np.zeros((height, width, 3), dtype=np.uint8)
    slice_img = frames_a[i]
    start_x = (width - slice_img.shape[1]) // 2
    frame[:, start_x:start_x + slice_img.shape[1], :] = slice_img
    flipped_frame = cv2.flip(frame, 1)
    video_writer.write(flipped_frame)

# Part 2: Transition sweep iterates over slices and progresses the sweep line
for step in range(transition_frames):
    slice_index = transition_idx + step
    if slice_index >= n_slices:
        break
    slice_a = frames_a[slice_index]
    slice_b = frames_b[slice_index]
    slice_width = slice_a.shape[1]
    start_x = (width - slice_width) // 2
    sweep_x = int(slice_width * step / (transition_frames - 1))

    frame = np.zeros((height, width, 3), dtype=np.uint8)
    # Left side: from image A, columns [0:sweep_x)
    print(f"sweep_x: {sweep_x}, {start_x}")
    frame[:, start_x : start_x + sweep_x, :] = frames_a[:, sweep_x:, :]

    # Right side: from image B, columns [sweep_x:end)
    frame[:, start_x + sweep_x : start_x + slice_width, :] = frames_b[:, :sweep_x, :]

    # Draw vertical yellow sweep line
    line_pos = start_x + sweep_x
    cv2.rectangle(frame, (line_pos, 0), (min(line_pos + line_width, start_x + slice_width - 1), height), line_color, thickness=-1)

    flipped_frame = cv2.flip(frame, 1)
    video_writer.write(flipped_frame)

# Part 3: Show remaining volume B slices after transition_frames
for i in range(transition_idx + transition_frames, n_slices):
    frame = np.zeros((height, width, 3), dtype=np.uint8)
    slice_img = frames_b[i]
    start_x = (width - slice_img.shape[1]) // 2
    frame[:, start_x:start_x + slice_img.shape[1], :] = slice_img
    flipped_frame = cv2.flip(frame, 1)
    video_writer.write(flipped_frame)

video_writer.release()
print(f"MP4 video saved to {output_video}")


In [ ]:
# resample 3T image to have same matrix size as 7T
import nibabel as nib
from nibabel.processing import resample_from_to
import numpy as np

def resample_to_same_matrix_but_keep_resolution(source_img, target_img):
    # Extract voxel sizes from source affine (scales are norms of affine matrix columns)
    voxel_sizes = np.sqrt(np.sum(source_img.affine[:3, :3] ** 2, axis=0))

    # Use target image shape but keep source voxel sizes
    target_shape = target_img.shape[:3]

    # Construct new affine matrix for target: diagonal matrix with voxel sizes and same origin as target
    # Extract target origin from target affine translation part
    target_origin = target_img.affine[:3, 3]

    # Build new affine with diagonal voxel sizes and target origin
    new_affine = np.diag(np.append(voxel_sizes, 1.0))
    new_affine[:3, 3] = target_origin

    # Resample source_img to new voxel space
    resampled_img = resample_from_to(source_img, (target_shape, new_affine))

    return resampled_img

# Usage

path_3T = "/host/verges/tank/data/daniel/3T7T/z/coregVols/PNE017_PX174/PX174a_0000.nii.gz"
path_7T = "/data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE017/ses-a1/anat/sub-PNE017_ses-a1_space-nativepro_T1w_brain.nii.gz" # path to first volume
path_3T_out = "/host/verges/tank/data/daniel/3T7T/z/coregVols/PNE017_PX174/PX174a_0000_resampled.nii.gz"

source_img = nib.load(path_3T)
target_img = nib.load(path_7T)
resampled_img = resample_to_same_matrix_but_keep_resolution(source_img, target_img)

# Save result if needed
nib.save(resampled_img, path_3T_out)


In [ ]:
# define directories
output_dir = "/host/verges/tank/data/daniel/3T7T/z/outputs"
values_dir = "values"
processed_output_dir = "/host/verges/tank/data/daniel/3T7T/z/outputs/fig_stats"


# 3T-7T ID correspondence
correps_IDs = {
    "path": "/host/verges/tank/data/daniel/3T7T/z/data/pt/IDs_ses_analyses_12Mar.csv",
    "3T_ID": "3T_ID",
    "7T_ID": "7T_ID",
    "3T_SES": "3T_SES",
    "7T_SES": "7T_SES"
}

#id_corresp = pd.read_csv(corresp_ID)

# Study names
MICs = {"name": "MICs"}

PNI = {"name": "PNI"}

#studies = ["MICs", "PNI"]

# zBrain analysis regions
cortex = {
    "region": "cortex",
    "surfaces": ["midthickness", "white"],
    "resolution": "32k",
    "features": ["ADC", "T1map", "volume"], # (list) features to extract
    #"smoothing": [10]
    "smoothing": [2,5,10]
}

hippocampus = {
    "region": "hippocampus",
    "surfaces": ["midthickness"],
    "resolution": "0p5mm",
    "features": ["ADC", "T1map", "volume"], # (list) features to extract
    #"smoothing": [5]
    "smoothing": [1,2,5]
}

subcortex = {
    "region": "subcortex",
    "features": ["ADC", "T1map", "volume"],
    "smoothing": [2,5,10]
}

regions = [cortex, hippocampus, subcortex]

In [ ]:
# get list of corresponding 3T, 7T aggregate files
files_lst = plots.corresp_paths(regions, MICs, PNI, output_dir, values_dir)
#print(files_lst)
shape = gen.lstOlst_shape(files_lst,print=False)
print(f"raw shape of files_lst (num files, num studies): {shape}")

# get missing files
missing = plots.get_missingPths(files_lst)

# remove missing files from list
for m in missing:
    files_lst.remove(m)

shape = gen.lstOlst_shape(files_lst,print=False)
print(f"shape of files_lst (num files, num studies): {shape}")

In [ ]:
print(files_lst)

In [ ]:
importlib.reload(vrtx)

# summary statistics & prepare for group hists
- All analysed PX vs all PNE for each file type


In [ ]:
# get summary stats for each file type
df_summary = pd.DataFrame()
clamp_files_lst = []
for lst in files_lst:
    clamp_lst = []
    for file in lst:
        print(os.path.basename(file))
        df = vrtx.summaryStats(file)
        df.insert(df.columns.get_loc("study") + 1, "region", os.path.dirname(file).split('/')[-1])
        
        df_summary = pd.concat([df_summary, df])

        # clamp values
        df_clamped = vrtx.clamp(file)
        
        if (df["study"] == "MICs").all():
            study = "MICs"
        elif (df["study"] == "PNI").all():
            study = "PNI"

        
        if (df["region"] == cortex).all():
            region = "cortex"
        elif (df["region"] == hippocampus).all():
            region = "hippocampus"
        elif (df["region"] == subcortex).all():
            region = "subcortex"
        
        clamp_name = os.path.basename(file).replace('.csv', '_clamp.csv')
        clamp_pth = os.path.join(output_dir,"values", study, region, "clamp", clamp_name)

        df_clamped.to_csv(clamp_pth, index=False)
        print(f"Clamped values saved to {clamp_pth}")
        clamp_lst.append(clamp_pth)
    clamp_files_lst.append(clamp_lst)

print(clamp_files_lst)

In [ ]:
out_pth = os.path.join(processed_output_dir, f"sumStats_{date}.csv")
df_summary.to_csv(out_pth, index=False)
print  (f"Summary stats saved to {out_pth}")

In [ ]:
out_pth = os.path.join(processed_output_dir, f"sumStats_{date}.csv")
df_summary.to_csv(out_pth, index=False)
print  (f"Summary stats saved to {out_pth}")

In [ ]:
#df_summary

In [ ]:
importlib.reload(plots)

In [ ]:
# group histograms

for files in clamp_files_lst:
    # check that base names are the same except for the study
    histName_mics = os.path.basename(files[0])
    histName_mics = histName_mics.split('_')

    histName_pni = os.path.basename(files[1])
    histName_pni = histName_pni.split('_')

    if histName_mics[1:] != histName_pni[1:]:
        print("Error: file names do not match. Skipping: \n\t%s\n\t%s" %(histName_mics, histName_pni))
        continue
    else:
        #print("File names match")
        pass

    hemi = histName_mics[1].split('-')[1]
    lbl = histName_mics[3].split('-')[1]
    feat = histName_mics[4].split('-')[1]
    smth = histName_mics[5].split('-')[1]
    #print(f"hemisphere: {hemi}, label: {lbl}, feature: {feat}, smoothing: {smth}")

    title = f"{feat}, smoothing: {smth} ({hemi}, {lbl})"
    #print(title)


    # plot histograms
    save_path = "/host/verges/tank/data/daniel/3T7T/z/outputs/fig_stats/hist_grp"
    save_name = f"grpHist_{feat}_smth-{smth}_{hemi}_{lbl}_{date}.png"
    save = os.path.join(save_path, save_name)
    fig = plots.group_hist(files, labels=[title, "MICs","PNI"], save_path=save)


In [ ]:
# Ridge plots (one line per participant)
# plot histogram

# check that file exists
# if not, continue to next file
for file in files_lst:
        
        
        # read in data
        df_mics = pd.read_csv(mics_file, index_col=False)
        df_pni = pd.read_csv(pni_file, index_col=False)
        # remove participants with Na in either df
        # df_mics = df_mics.dropna()
        # df_pni = df_pni.dropna()
        
        # keep only overlapping participants both dfs
        
        ## need to remap col names according to 3T-7T ID correspondence

        ## keep only overlapping columns
        # cols = df_mics.columns.intersection(df_pni.columns)
        # df_mics = df_mics[cols]
        # df_pni = df_pni[cols]

        ## Take histogram for each participant

        # print(df_mics.head())
        break
        # # construct histogram
        # fig = plots.ridge(df_mics, matrix_df = df_mics)
        # # show histogram
        # fig.show()

In [ ]:

        print(path)
        
        df = pd.read_csv(path)
        fig = plots.histStack(df)
        # display plot
        fig.show()